In [ ]:
import pandas as pd 
import numpy as np
import os
from openpyxl import load_workbook
index_hopital = pd.read_excel('C:\\Users\\amrou\\Downloads\\code_hopital.xlsx')
index_cliniques = pd.read_excel('C:\\Users\\amrou\\Downloads\\all_Clin1.xlsx')
a = []
for j in range (len(index_hopital)) :
    a.append(str(index_hopital.Type[j]) + str(int(index_hopital.Code[j])))
index_hopital["Index"]=a
base=pd.DataFrame()
base1=pd.DataFrame()

In [ ]:
def base_Hp_Clin (directory) :
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".xlsx"):
                 s = os.path.join(root, file)
                 paths.append(s)
   
    
    all_data = pd.DataFrame()
   
    
    name=paths
    for f in (paths) :
        name = f
        name=name.replace(".xlsx","").replace("_Etat_Occupation_ Lits_Covid_Répartition-malades","").replace(directory,"")[7:18].lstrip().rstrip()
        df = pd.read_excel(f, skiprows = 1).assign(Date=name)
        
        list =["Gd Tunis","Nombre Total des malades hospitalisés Covid +","Nd-Est","Nd Ouest","C. Est","C. Ouest","S. Ouest","S. Est","Total "]
   
        for i in range (len(list)) :   
             
            df.drop(df[df["Gouvernorat"] == list[i]].index, inplace = True)
        
        
        
        if (len(df.columns)==19) :
            
            df.columns=["Gov","Struct","lits_rea","lits_rea_occ","lits_rea_disp","mal_conf","mal_intu","lits_o2","lits_o2_occ","lits_o2_disp","mal_conf","urg","mal_intu","z_tamp","rq","pt_foc","tel","mail","Date"]
        
            df=df.drop(["mal_intu","mal_conf","urg","rq","pt_foc","tel","mail"],axis=1)
        else :
            df.columns=["Gov","Struct","lits_rea","lits_rea_occ","lits_rea_disp","mal_intu","lits_o2","lits_o2_occ","lits_o2_disp","urg","mal_intu","z_tamp","rq","pt_foc","tel","mail","Date"]
            df=df.drop(["mal_intu","urg","rq","pt_foc","tel","mail"],axis=1)        
        
        df['Struct'].fillna('remove', inplace=True)
        df.drop(df[df.Struct == 'Nombre de places déclarées via plateforme' ].index, inplace=True)
        df.drop(df[df.Struct == 'Nombre de places déclarées via mail ou fax ou tel' ].index, inplace=True)
        df.drop(df[df.Gov == 'Nombre Total des malades hospitalisés Covid +' ].index, inplace=True)
        df.drop(df[df.Gov == 'Total ' ].index, inplace=True)
        df.drop(df[df.Struct == 'remove' ].index, inplace=True)
        df.dropna(how='all')
    
    
        df=df.set_index("Gov")

        for j in range(len(df.Struct)) :
            df['Struct'][j] = df['Struct'][j].upper()
    
        df["INDEX"]=""
        for i in range(len(df["Struct"])) :
            for j in range(len(index_hopital["Struct"])):
                if df["Struct"][i]==index_hopital["Struct"][j] :
                    df["INDEX"][i]=index_hopital["Index"][j]
    
    
        df=df.reset_index()
    
        gov = df['Gov'][0]
    
        for j in range(len(df.Struct)) :
            df['Struct'][j] = df['Struct'][j].upper()
        
            if (df['Gov'][j] != gov) and (str(df['Gov'][j]) =='nan')   :
                df['Gov'][j] = gov
               
            else :
                gov = df['Gov'][j]
               
    
    
    
        df=df.loc[:,["Date","Gov","Struct","INDEX","lits_rea","lits_rea_occ","lits_rea_disp","lits_o2","lits_o2_occ","lits_o2_disp","z_tamp"]]
        
    
    
        all_data = all_data.append(df)
    all_data[["lits_rea","lits_rea_occ","lits_rea_disp","lits_o2","lits_o2_occ","lits_o2_disp","z_tamp"]]=all_data[["lits_rea","lits_rea_occ","lits_rea_disp","lits_o2","lits_o2_occ","lits_o2_disp","z_tamp"]].fillna(0).abs()
    
    
    
    ############### Pour creation de la base totale######################
    
    writer = pd.ExcelWriter('Hopit_Clin.xlsx')
    all_data.to_excel(writer,'Hopitaux', index = False)
    writer.save()
    ###############pour ajouter de nouveau fichiers ####################
   
    #book = load_workbook('Hopit_Clin.xlsx')
    #writer = pd.ExcelWriter('Hopit_Clin.xlsx', engine='openpyxl',mode='a') 
    #writer.book = book

    #writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #all_data.to_excel(writer, "Hopitaux",index = False,startrow=len(base)+1,header=False)

    #writer.save()
    
        
################################################## CLINIQUES ##################################################################
    
    for i in range(len(index_cliniques.Clin)) :
        index_cliniques['Clin'][i] = index_cliniques['Clin'][i].upper()
        
    all_data1 = pd.DataFrame()
    for f in paths:
        name = f
        name=name.replace(".xlsx","").replace("_Etat_Occupation_ Lits_Covid_Répartition-malades","").replace(directory,"")[7:18].lstrip().rstrip()
    
        df = pd.read_excel(f,'Cliniques', skiprows = 1).assign(Date=name)
       
            
        if len(df.columns)== 14 :
            df.columns=["Gov","Clin","lits_o2","lits_o2_occ","lits_o2_disp","lits_rea","lits_rea_occ","lits_rea_disp","mal_intu","urg","nn","nan","non","Date"]
            df=df.drop(["lits_o2","lits_o2_disp","lits_rea","lits_rea_disp","urg","nn","nan","non"],axis=1)
        elif len(df. columns) == 10 :
            df.columns=["Gov","Clin","lits_o2_occ","lits_rea_occ","mal_intu","urg","nn","nan","non","Date"]
            df=df.drop(["urg","nn","nan","non"],axis=1)
        else :
            df.columns=["Gov","Clin","lits_o2_occ","lits_rea_occ","mal_intu","urg","Date"]
            df=df.drop(["urg"],axis=1)
        df['Clin'].fillna('remove', inplace=True)
        df.drop(df[df.Gov == 'Nombre de places déclarées via plateforme' ].index, inplace=True)
        df.drop(df[df.Gov == 'Nombre de places déclarées via mail ou fax ou tel' ].index, inplace=True)
        df.drop(df[df.Gov == 'Nombre Total de Malades hospitalisés' ].index, inplace=True)
        df.drop(df[df.Gov == 'TOTAL' ].index, inplace=True)
        df.drop(df[df.Clin == 'remove' ].index, inplace=True)
        df.dropna(how='all')
        df=df.reset_index()
    
        gov = df['Gov'][0]
    
        for j in range(len(df.Clin)) :
            df['Clin'][j] = df['Clin'][j].upper()
        
            if (df['Gov'][j] != gov) and (str(df['Gov'][j]) =='nan')   :
                df['Gov'][j] = gov
               
            else :
                gov = df['Gov'][j]
            
    
        df3 = pd.merge(df, index_cliniques, on="Clin")
        
        for i in  (df3.Clin) :
            if i not in index_cliniques.Clin :
                k=df3[df3['Clin']==i].index.tolist()
    
        
        all_data1 = all_data1.append(df3) 
    all_data1=all_data1.loc[:,["Date","Gov","Clin","Index","lits_o2_occ","lits_rea_occ","mal_intu"]]
    all_data1[["lits_o2_occ","lits_rea_occ","mal_intu"]]=all_data1[["lits_o2_occ","lits_rea_occ","mal_intu"]].fillna(0).abs()
    
    
    
    ###################pour_creation_de_la_base_1ere_fois#########    
    
    with pd.ExcelWriter('Hopit_Clin.xlsx',mode='a',engine='openpyxl') as writer:  
        all_data1.to_excel(writer, sheet_name='Cliniques',index=False)
    writer.save()
    return(all_data,all_data1)
    
    

    #################pour ajouter de nouveaux fichiers #########

    #writer = pd.ExcelWriter('Hopit_Clin.xlsx', engine='openpyxl',mode='a') 
    #writer.book = book

    #writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #all_data1.to_excel(writer, "Cliniques",index = False,startrow=len(base1)+1,header=False)
    
    #writer.save()
    #writer.close()
    
    #return(all_data,all_data1)    

In [ ]:
baseH,baseC=base_Hp_Clin("C:\\Users\\amrou\\Downloads\\BD")
base=base.append(baseH)
base1=base1.append(baseC)

<ipython-input-9-8ff80c638aa3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["INDEX"][i]=index_hopital["Index"][j]
<ipython-input-9-8ff80c638aa3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:65: SettingWithCopyWarning: 
A value is 

<ipython-input-9-8ff80c638aa3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:54: SettingWithCopyWarning: 
A value is trying to be set on a 

<ipython-input-9-8ff80c638aa3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:54: SettingWithCopyWarning: 
A value is trying to be set on a 

<ipython-input-9-8ff80c638aa3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:54: SettingWithCopyWarning: 
A value is trying to be set on a 

<ipython-input-9-8ff80c638aa3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-9-8ff80c638aa3>:54: SettingWithCopyWarning: 
A value is trying to be set on a 

...

<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy

<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy

<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy

<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy

<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gov'][j] = gov
<ipython-input-9-8ff80c638aa3>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clin'][j] = df['Clin'][j].upper()
<ipython-input-9-8ff80c638aa3>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [ ]:
baseH.apply(lambda s: pd.to_numeric (s,errors='coerce').notnull().all())

Date             False
Gov              False
Struct           False
INDEX            False
lits_rea          True
lits_rea_occ      True
lits_rea_disp     True
lits_o2           True
lits_o2_occ       True
lits_o2_disp      True
z_tamp            True
dtype: bool

In [ ]:
baseC.apply(lambda s: pd.to_numeric (s,errors='coerce').notnull().all())

Date            False
Gov             False
Clin            False
Index           False
lits_o2_occ      True
lits_rea_occ     True
mal_intu         True
dtype: bool

In [ ]:
#def ajout_nouveau(Ndirectory) :
 #   baseH,baseC=base_Hp_Clin(Ndirectory)
  #  return(baseH,baseC)

In [ ]:
#baseH,baseC=ajout_nouveau("C:\\Users\\amrou\\Downloads\\NBD")
#base=base.append(baseH)
#base1=base1.append(baseC)

<ipython-input-2-8ff80c638aa3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-2-8ff80c638aa3>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["INDEX"][i]=index_hopital["Index"][j]
<ipython-input-2-8ff80c638aa3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Struct'][j] = df['Struct'][j].upper()
<ipython-input-2-8ff80c638aa3>:65: SettingWithCopyWarning: 
A value is 